# 20 - Entity linking with NIL option : example of the plot natures
* This nobook propose methods to perform entity linking between historical tables cells and existing entities in a domain KG.
* Developped with the example of land plots natures (already 160 natures in the ontology).

In [1]:
import pandas as pd
import rdflib
from rdflib.namespace import SKOS, RDF, RDFS
import glob

In [2]:
import sys
import os
# Access to the utils directory
current_dir = os.getcwd()
utils_dir = os.path.join(current_dir, '..', 'utils')
sys.path.append(utils_dir)

## 1. Retrieve candidates from the domain ontology

In [3]:
from rdf import skosConceptUtil

CSV_PATH = '/home/STual/DAN-cadastre/data/ONTO/SKOS_NATURE.csv'
skos_graph = skosConceptUtil()
skos_graph.init_nature_concepts_from_csv(CSV_PATH,'/home/STual/DAN-cadastre/data/ONTO/')

In [4]:
# Create a graph
g = rdflib.Graph()

# Parse the Turtle file into the graph
g.parse("/home/STual/DAN-cadastre/data/ONTO/natures.ttl", format="ttl")

# Define a SPARQL query that uses the vocabularies as variables
query = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?subject ?label
WHERE {
  ?subject a skos:Concept .
  {?subject skos:prefLabel ?label .} UNION {?subject skos:altLabel ?label .} UNION {?subject skos:hiddenLabel ?label .}
  FILTER(LANG(?label) = "fr")
}
"""
# Execute the query on the graph
results = g.query(query)

results_ls = []
# Print the results
for row in results:
    results_ls.append([str(row.subject),str(row.label)])

In [5]:
results_ls[0:5]

[['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Abreuvoir',
  'Abreuvoir'],
 ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Abricotier',
  "Plantation d'abricotiers"],
 ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Abricotier',
  'Abricotiers'],
 ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Acacia',
  "Plantation d'acacias"],
 ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Acacia',
  'Acacias']]

In [6]:
labels_ = [x[1] for x in results_ls]
print(labels_)

['Abreuvoir', "Plantation d'abricotiers", 'Abricotiers', "Plantation d'acacias", 'Acacias', 'Ajonc', 'Ajoncs', 'Allée', "Plantation d'amandiers", 'Amandiers', 'Appenti', 'Ardoisière', 'Ardoise', "Carrière d'ardoise", 'Argillière', 'Argile', "Carrière d'argile", "Plantation d'asperges", 'Asperge', 'Atelier', 'Aulnaie', 'Baraque', 'Bassin', 'Bâtiment rural', 'Bâtiments ruraux', 'Batterie', 'Batterie de côte', 'Bief', "Bois d'agrément", 'Brousaille', 'Bruyère', 'Bruyères', 'Buissières', "Canal d'agrément", 'Plantation de cassis', 'Plantation de cédratier', 'Cédratier', 'Cerisaies', 'Cerise', 'Plantation de cerises', 'Chantier', 'Dépôt de marchandises', 'Charmille', 'Allée couverte', 'Châtaigneraies', 'Chemin', 'Chemin de fer', 'Chenevière', 'Champ de chanvre', 'Chanvre', 'Cimetiere', 'Plantation de citronnier', 'Citron', 'Clos', 'Cour commune', "Cours d'eau", 'Courtil', 'Crayère', 'Carrière de craie', 'Craie', 'Cressonnière', 'Cresson', 'Cuisine', 'Dépendance', 'Digue', 'Douve', 'Dune', '

## 2. Retrieve mentions

In [7]:
#FOLDER = "/home/STual/DAN-cadastre/inference/training120325_config2025_prod_2000epochs-DEV"
FOLDER = "/home/STual/DAN-cadastre/inference/LHAY"
JSONS = glob.glob(FOLDER + '/*.json')
print(len(JSONS))

26


* *Idem* and *ditto* have to be replaced by their concrete value (at the page level)

In [8]:
import json
import glob
from string_utils import TableValuesPostTreatment

chars_to_split = ["Ⓐ", "Ⓑ", "Ⓒ", "Ⓓ", "Ⓔ", "Ⓕ", "Ⓖ"]

# Extract cell contents from line transcription
#TableValuesPostTreatment.update_jsons_with_objects_details(FOLDER, ["Ⓐ", "Ⓑ", "Ⓒ", "Ⓓ", "Ⓔ", "Ⓕ", "Ⓖ"])

[nltk_data] Downloading package punkt to /home/STual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/STual/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
KEY = "Ⓔ"
IX = 4
distinct_natures = []
#for j in JSONS:
with open(JSONS[IX]) as f:
    page_name = JSONS[IX].replace(FOLDER+'/','').replace('.json','')
    print(page_name)
    page = json.load(f)
    cells = page["entities"]
    
    for c in cells:
        keys = c.keys()
        if KEY in keys:
            print(c[KEY])
            distinct_natures.append(c[KEY]['interpreted_text'])
        else:
            print('MISSING')
distinct_natures = list(set(distinct_natures))
distinct_natures = [d for d in distinct_natures if d is not None]
print(distinct_natures)

9e52f7f9-0e66-4753-bcf3-d3cb0916678c
{'text': 'Terre', 'offset': 31, 'length': 5, 'interpreted_text': 'Terre', 'uris': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Terre']}
{'text': 'T', 'offset': 33, 'length': 1, 'interpreted_text': 'T', 'uris': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Terre']}
{'text': 'T', 'offset': 33, 'length': 1, 'interpreted_text': 'T', 'uris': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Terre']}
{'text': 'T', 'offset': 25, 'length': 1, 'interpreted_text': 'T', 'uris': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Terre']}
{'text': 'T.', 'offset': 37, 'length': 2, 'interpreted_text': 'T.', 'uris': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Terre']}
{'text': 'T.p.', 'offset': 31, 'length': 4, 'interpreted_text': 'T.p.', 'uris': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/TerrePlantee']}
{'text': 'T', 'offset': 28, 'length': 1, 'interpreted_text

## 3. Entity linking
* Entity-linking has to consider NIL entity linking as all the mentions in historical tables have no corresponding concept in the ontology.

### 3.1 Edit distance based similarities

#### Gestalt pattern matching (Difflib)

In [10]:
from linking_utils import EditDistanceBasedSimilarity

remove_chars_regex = '→()↑↓×.,!?;:-@#$%^&*'
replacement_char = ' '
split_chars = '→, .;'
matched_natures =  EditDistanceBasedSimilarity.EL_gestaltpatternmatching(distinct_natures, labels_, remove_chars_regex, replacement_char, split_chars, top_k=1, threshold=0.7)

/home/STual/.venv/venv_sti/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Queries : ['T. pt', 'T.pl', 'T', 'Terre', 'T.', 'T.p.']

Query T. pt normalized as t pt
Step 1 results : [(199, 'T pt', 0.8888888888888888)]
Query T.pl normalized as t pl
Step 1 results : [(199, 'T pt', 0.75)]
Query T normalized as t
Step 1 results : [(194, 'T', 1.0)]
Query Terre normalized as terre
Step 1 results : [(193, 'Terre', 1.0)]
Query T. normalized as t
Step 1 results : [(194, 'T', 1.0)]
Query T.p. normalized as t p
Step 1 results : [(199, 'T pt', 0.8571428571428571)]


### 3.2 Embedding similarity

In [11]:
from linking_utils import LinkingUtils, EmbeddingSimilarity
from sentence_transformers import SentenceTransformer

In [12]:
from linking_utils import LinkingUtils, EmbeddingSimilarity
from sentence_transformers import SentenceTransformer

embedder_name = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(embedder_name)

remove_chars_regex = '→()↑↓×.,!?;:-@#$%^&*'
replace_regex = "→()↑↓×."
split_regex = "., "

embedding_results = EmbeddingSimilarity.EL_embedding_cosinedistance_matching(distinct_natures, labels_, embedder, remove_chars_regex, replacement_char, split_chars, top_k=1, threshold=0.8)
print(embedding_results)

Queries : ['T. pt', 'T.pl', 'T', 'Terre', 'T.', 'T.p.']

Query T. pt normalized as t  pt

Query: t  pt
Top 1 most similar concepts:
199 T pt (Score: 1.0000)
Step 1 results : [(199, 'T pt', 1.0)]
Query T.pl normalized as t pl

Query: t pl
Top 1 most similar concepts:
194 T (Score: 0.7484)
Step 1 results : []
NEXT
Try to match subqueries ['pl']

Query: pl
Top 1 most similar concepts:
152 Plage (Score: 0.5657)
Query T normalized as t

Query: t
Top 1 most similar concepts:
194 T (Score: 1.0000)
Step 1 results : [(194, 'T', 1.0)]
Query Terre normalized as terre

Query: terre
Top 1 most similar concepts:
193 Terre (Score: 1.0000)
Step 1 results : [(193, 'Terre', 1.0)]
Query T. normalized as t 

Query: t 
Top 1 most similar concepts:
194 T (Score: 1.0000)
Step 1 results : [(194, 'T', 1.0)]
Query T.p. normalized as t p 

Query: t p 
Top 1 most similar concepts:
200 T. p. (Score: 0.8698)
Step 1 results : [(200, 'T. p.', 0.869800329208374)]
[[(199, 'T pt', 1.0)], [(1000000, 'NIL', 0.0)], [(194, 

## 4. Add results to jsons

In [13]:
from linking_utils import LinkingUtils

uris_values = LinkingUtils.associate_matches_with_values(distinct_natures, matched_natures, results_ls, 0)
uris_values

{'T. pt': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/TerrePlantee'],
 'T.pl': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/TerrePlantee'],
 'T': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Terre'],
 'Terre': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Terre'],
 'T.': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/Terre'],
 'T.p.': ['http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/TerrePlantee']}

## 5. Main

In [14]:
JSONS = sorted(glob.glob(FOLDER + '/*.json'))
KEY = "Ⓔ"
METHOD = "editdistancebased"

for JSON in JSONS:
    print()
    distinct_natures = []
    with open(JSON) as f:
        page_name = JSON.replace(FOLDER,'').replace('.json','').replace('/','')
        print(page_name)
        page = json.load(f)
        cells = page["entities"]
        
        for c in cells:
            keys = c.keys()
            if KEY in keys:
                distinct_natures.append(c[KEY]['interpreted_text'])
                
    distinct_natures = list(set(distinct_natures))
    distinct_natures = [d for d in distinct_natures if d is not None]
    
    remove_chars_regex = '→()↑↓×.,!?;:-@#$%^&*±'
    replacement_char = ' '
    split_chars = '→, .;'
    if METHOD == "embeddingbased":
        embedding_results = EmbeddingSimilarity.EL_embedding_cosinedistance_matching(distinct_natures, labels_, embedder, remove_chars_regex, replacement_char, split_chars, top_k=1, threshold=0.8)
    elif METHOD == "editdistancebased":
        matched_natures =  EditDistanceBasedSimilarity.EL_gestaltpatternmatching(distinct_natures, labels_, remove_chars_regex, replacement_char, split_chars, top_k=1, threshold=0.7)
    
    uris_values = LinkingUtils.associate_matches_with_values(distinct_natures, matched_natures, results_ls, 0)
    for c in page["entities"]:
        keys = c.keys()
        if KEY in keys :
            if c[KEY]["interpreted_text"] is not None:
                c[KEY]['uris'] = uris_values[c[KEY]["interpreted_text"]]
        
    with open(JSON, 'w') as f:
        json.dump(page, f, indent=4)


064cc4ee-9d5e-4922-a6dc-954aaf38fab2
Queries : ['T']

Query T normalized as t
Step 1 results : [(194, 'T', 1.0)]

26d52fc2-aafc-4e58-91f8-68bd6851ad37
Queries : ['T']

Query T normalized as t
Step 1 results : [(194, 'T', 1.0)]

2c59c9c6-5ed5-4fc1-85cd-4d162c324239
Queries : ['Tpt', 'T', 'Terre', 'T.']

Query Tpt normalized as tpt
Step 1 results : [(199, 'T pt', 0.8571428571428571)]
Query T normalized as t
Step 1 results : [(194, 'T', 1.0)]
Query Terre normalized as terre
Step 1 results : [(193, 'Terre', 1.0)]
Query T. normalized as t
Step 1 results : [(194, 'T', 1.0)]

2e198618-c734-4d73-a67f-79fda49504f5
Queries : ['bis', 'T', 'Terre', 'big']

Query bis normalized as bis
Step 1 results : [(223, 'Bois', 0.8571428571428571)]
Query T normalized as t
Step 1 results : [(194, 'T', 1.0)]
Query Terre normalized as terre
Step 1 results : [(193, 'Terre', 1.0)]
Query big normalized as big
Step 1 results : []
NEXT
Try to match subqueries ['big']

3435dedc-ddc0-4dfd-b834-d04148e44ddf
Queries : ['